In [14]:
import numpy as np
import math

from itertools import product

We need to find a triple $x,y,z \in \mathbb{N}$ such that $x > y > z > 0$ and
1. $x+y = a^2,$
2. $x-y = b^2,$ 
3. $x+z = c^2,$
4. $x-z = d^2,$ 
5. $y+z = m^2,$
6. $y-z = n^2$

$$
\begin{align*}
c^2 &= b^2 + m^2 & (2+5) \\
d^2 &= b^2 + n^2 & (2+6) \\
a^2 &= c^2 + n^2 & (3+6) \\
a^2 &= d^2 + m^2 & (4+5) \\
\\
\end{align*}
$$

From these equations, we can think of the relationships as right triangles (Pythagorean theorem). By drawing them, in some orientation where we can "match" the two equations with $a$ in them and also the two equations with $b$ in them. We can to solve this problem with not only Pythagorean triples (like the four equations above) but also one Pythagorean quadruple if we combine them to $a^2 = b^2 + m^2 + n^2$.

From here the algorithm just basically came from realizing $c > d$, and that each pair $(a,b), (c,d),$ and $(m,n)$ must have the same parity, which allowed faster pruning. I generated the pythagorean triples (not just primitives) up to $10^4$ (arbitrary, just guessing a small number since the result is supposed to be minimized). To speed up the algorithm further, I used dictionaries that stored for each hypotenuse the associated legs, and each leg the associated other leg and hypotenuse since there were a lot of lookups. In particular, we needed $c$ an $d$ 

Also, I noticed that the triangles can after a little bit of pen and paper drawing, I then saw that the four triangles make up a tetrahedron, with point $(m,0,0), (0,n,0), (0,0,b), (m,n,0),$ and (0,0,0). Looking up the shape of such tetrahedra led me to [Euler bricks][1] (in particular, the space diagonal and almost-perfect cuboid sections) and [Heronian triangles][2]. The problem here is identical to find solutions to face cuboids. I couldn't find a formula to generate these directly, but it was an interesting explroation.

[1]: https://en.wikipedia.org/wiki/Euler_brick
[2]: https://en.wikipedia.org/wiki/Heronian_triangle

In [58]:
def gcd(a,b):
    if a == 0:
        return b
    return gcd(b % a, a)

# Euclid's formula
def pythagorean_triples(max_perimeter):
    if max_perimeter < 12:
        return []

    triples = []
    to_check = [(2, 1)]
    seen = set()
    while to_check:
        m,n = to_check.pop(0)
        if (m,n) in seen: continue

        seen.add((m,n))
        a,b,c = m*m - n*n, 2*m*n, m*m + n*n

        if a+b+c >= max_perimeter: continue

        # require gcd = 1 for primitive triples
        if gcd(m,n) == 1 and (m + n) % 2 == 1:
            a,b,c = min(a,b), max(a,b), c
            triples.append((a,b,c))
            
            # do a sieve to get remaining triangles from this primitive
            k = 2
            while sum(triples[-1]) < max_perimeter:
                triples.append((k*a, k*b, k*c))
                k += 1

        if m > n + 1:
            to_check.append((m, n+1))
        to_check.append((m+1, n))

    return triples

In [59]:
trips = pythagorean_triples(10**4)
hyps = {}
legs = {}

for trip in trips:
    leg1, leg2, hyp = trip

    hyps[hyp] = hyps.get(hyp, []) + [(leg1, leg2)]
    legs[leg1] = legs.get(leg1, []) + [(leg2, hyp)]
    legs[leg2] = legs.get(leg2, []) + [(leg1, hyp)]

In [60]:
hs = []
for h in hyps:
    # c and d need to be both hypotenuses and legs of triangles
    if len(hyps[h]) > 1 and h in legs:
        hs.append(h)

hs.sort()

In [61]:
# we know c and d are hypotenuses and also legs of another triangle
# we also know d < c since x+y = c^2 and x-y = d^2 and x,y > 0
# and also, we get x = (c^2 + d^2)/2 so c and d must have the same parity
sols = []
for i in range(len(hs)):
    # i such that hyps[hs[i]] = d
    # we know b^2 + n^2 = d^2
    if i % 10000 == 0:
        print(i)
    d = hs[i]
    bnds = hyps[d]
    for j in range(i+1, len(hs)):
        # j such that hyps[hs[j]] = c
        # we know b^2 + m^2 = c^2
        c = hs[j]
        bmcs = hyps[c]

        if (c+d) % 2:
            continue

        for (ld1, ld2), (lc1, lc2) in product(bnds, bmcs):
            b = -1
            # see if they have any legs in common, which can be side b
            if ld1 == lc1:
                b, n, m = ld1, ld2, lc2
            elif ld1 == lc2:
                b, n, m = ld1, ld2, lc1
            elif ld2 == lc1:
                b, n, m = ld2, ld1, lc2
            elif ld2 == lc2:
                b, n, m = ld2, ld1, lc1
            
            if b < 0:
                continue

            for (ld, ad), (lc, ac) in product(legs[d], legs[c]):
                if ad != ac or ld != m or lc != n:
                    continue
                
                # if all conditions here are true, then a = ac = ad
                # so a, b, c, d, m, n
                sols.append((ac, b, c, d, m, n))
                print(ac, b, c, d, m, n)


0
1394 208 1360 370 1344 306
925 117 765 533 756 520
4225 399 4199 615 4180 468
2165 333 2067 725 2040 644
2788 416 2720 740 2688 612
2210 896 2146 1040 1950 528
1850 234 1530 1066 1512 1040
4182 624 4080 1110 4032 918
4330 666 4134 1450 4080 1288
2775 351 2295 1599 2268 1560
3700 468 3060 2132 3024 2080
2665 1881 2431 2175 1540 1092
3485 1771 3179 2275 2640 1428
3965 3627 3875 3723 1364 840


In [62]:
min_sol_combo = (1394, 208, 1360, 370, 1344, 306)
min_sol = 2799586
for sol in sols:
    a, b, c, d, m, n = sol

    x = (a*a + b*b)//2
    y = (m*m + n*n)//2
    z = (c*c - d*d)//2

    if x + y + z < min_sol:
        min_sol = x + y + z
        min_sol_combo = sol

a, b, c, d, m, n = min_sol_combo
x = (a*a + b*b)//2
y = (m*m + n*n)//2
z = (c*c - d*d)//2

print(x,y,z)

print(math.sqrt(x+y))
print(math.sqrt(x-y))
print(math.sqrt(x+z))
print(math.sqrt(x-z))
print(math.sqrt(y+z))
print(math.sqrt(y-z))

print(x+y+z)

434657 420968 150568
925.0
117.0
765.0
533.0
756.0
520.0
1006193
